<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

* Импортируем нужные библиотеки и их функции
* Откроем файл, посмотрим его данные
* Проверим данные на дупликаты и пропуски

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.metrics import (
    accuracy_score, confusion_matrix, f1_score, roc_auc_score, 
    recall_score, precision_score
)

In [2]:
try:
    data = pd.read_csv('/datasets/Churn.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/Churn.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Удалим первые 3 столбца, так как они нам не понадобятся

In [4]:
data = data.iloc[:,3:]

In [5]:
data.duplicated().sum()

0

Дубликатов не обнаружено

In [6]:
data.isna().sum()

CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

* Замечаем пропуски в столбце 'Tenture'
* Заменим их рандомными значениями 

In [7]:
data['Tenure'].describe()

count    9091.000000
mean        4.997690
std         2.894723
min         0.000000
25%         2.000000
50%         5.000000
75%         7.000000
max        10.000000
Name: Tenure, dtype: float64

In [8]:
data['Tenure'] = data['Tenure'].mask(data['Tenure'].isna(), 
                                   np.random.uniform(1, 10, size=len(data)))

In [9]:
data['Tenure'].describe()

count    10000.000000
mean         5.043074
std          2.873750
min          0.000000
25%          3.000000
50%          5.000000
75%          7.471693
max         10.000000
Name: Tenure, dtype: float64

Посмотрим какие данные встречаются в столбцах 'Gender' и 'Geography'

In [10]:
data['Gender'].value_counts()

Male      5457
Female    4543
Name: Gender, dtype: int64

In [11]:
data['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

Перекодируем категориальные признаки с помощью OHE

In [12]:
data_ohe = pd.get_dummies(data, columns = ['Gender', 'Geography'], drop_first=True)

Разделим данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1

In [13]:
features = data_ohe.drop('Exited', axis=1)
target = data_ohe['Exited']

In [14]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

In [15]:
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(6000, 11)
(2000, 11)
(2000, 11)


Проведем масштабирование количественных признаков

In [16]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary', 
           'NumOfProducts']

scaler = StandardScaler()
scaler.fit(features_train[numeric]) 

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

## Исследование задачи

* Выберем лучшую модель
* Подберем оптимальные гиперпараметры

In [17]:
best_model = None
best_result = 0
best_depth = 0
best_auc_roc = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid)
    result = f1_score(target_valid, predictions) 
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        best_auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('DecisionTreeClassifier')
print('best_depth:', best_depth)
print('f1_score:', best_result)
print('auc_roc:', best_auc_roc)

DecisionTreeClassifier
best_depth: 6
f1_score: 0.5722983257229833
auc_roc: 0.8175406335629903


In [18]:
best_f1 = 0
best_depth = 0
best_auc_roc = 0
best_estimators = 0

for estimators in range(150, 171, 10):    
    for depth in range(12, 30, 10):    
        model = RandomForestClassifier(random_state=12345, max_depth=depth, 
                                       n_estimators=estimators)
        model.fit(features_train, target_train)
        predicted = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted) 
        if f1 > best_f1:
            best_f1 = f1
            best_depth = depth
            best_estimators = estimators
        
probabilities = model.predict_proba(features_valid)
probabilities_one = probabilities[:,1]
best_auc_roc = roc_auc_score(target_valid, probabilities_one) 
            
print('RandomForestClassifier')
print('best depth:', best_depth)
print('best n_estimators:', best_estimators)
print('f1_score:', best_f1)
print('auc_roc:', best_auc_roc)

RandomForestClassifier
best depth: 22
best n_estimators: 170
f1_score: 0.5794947994056464
auc_roc: 0.8364744826668442


По результатам исследования лучшей моделью оказалась RandomForestClassifier

## Борьба с дисбалансом

Сначала попробуем убрать дисбаланс с помощью аргумента class_weight='balanced'

In [19]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, 
                                   class_weight='balanced') 
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid)
    result = f1_score(target_valid, predictions) 
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth

        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]

        best_auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('DecisionTreeClassifier')
print('best_depth:', best_depth)
print('f1_score:', best_result)
print('auc_roc:', best_auc_roc)

DecisionTreeClassifier
best_depth: 5
f1_score: 0.5957446808510639
auc_roc: 0.8317918690531638


In [20]:
model = RandomForestClassifier(
    random_state=12345, max_depth = 12, n_estimators = 170, 
    class_weight='balanced')

model.fit(features_train, target_train) 
predictions = model.predict(features_valid)
result = f1_score(target_valid, predictions)

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('RandomForestClassifier')
print('f1_score:', result)
print('auc_roc:', auc_roc)

RandomForestClassifier
f1_score: 0.6119205298013245
auc_roc: 0.8487923348193491


Увеличим количество редких признаков в 4 раза

In [21]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

model = DecisionTreeClassifier(random_state=12345, max_depth=5)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid) 

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('f1_score:', f1_score(target_valid, predicted_valid))
print('auc_roc:', auc_roc)

f1_score: 0.5957446808510639
auc_roc: 0.8317918690531638


In [22]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

model = RandomForestClassifier(
    random_state=12345, max_depth = 12, n_estimators = 170, 
    class_weight='balanced')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid) 

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('f1_score:', f1_score(target_valid, predicted_valid))
print('auc_roc:', auc_roc)

f1_score: 0.6157407407407407
auc_roc: 0.8472559112987619


Уменьшим количество частых признаков до 30%

In [23]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + 
        [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + 
        [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, 
                                                      target_train, 0.3)

model = DecisionTreeClassifier(random_state=12345, max_depth=5)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid) 

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('f1_score:', f1_score(target_valid, predicted_valid))
print('auc_roc:', auc_roc)

f1_score: 0.579212916246216
auc_roc: 0.8141109007434112


In [24]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + 
        [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + 
        [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, 
                                                      target_train, 0.3)

model = RandomForestClassifier(
    random_state=12345, max_depth = 12, n_estimators = 170, 
    class_weight='balanced')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid) 

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

print('f1_score:', f1_score(target_valid, predicted_valid))
print('auc_roc:', auc_roc)

f1_score: 0.5978152929493545
auc_roc: 0.8438866675941664


Наилучший результат достигается с помощью  'взвешивания классов'

## Тестирование модели

Лучшей моделью является RandomForestClassifier cо следующими гиперпараметрами: глубина - 12, количество деревьев - 170, наилучший результат f1 модель достигла при помощи взвешивания классов, так что будем тестировать её.

In [25]:
model = RandomForestClassifier(
    random_state=12345, max_depth = 12, n_estimators = 170, class_weight='balanced')

model.fit(features_train, target_train) 
predictions = model.predict(features_valid)
result = f1_score(target_valid, predictions)

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]

auc_roc = roc_auc_score(target_test, probabilities_one_test) 

print('f1_score:', result)
print('auc_roc:', auc_roc)
print('recall_score:', recall_score(target_valid, predictions))
print('precision_score:', precision_score(target_valid, predictions))
print('accuracy_score:', accuracy_score(target_valid, predictions))

f1_score: 0.6119205298013245
auc_roc: 0.8558099512645579
recall_score: 0.5526315789473685
precision_score: 0.685459940652819
accuracy_score: 0.8535


Сравним результаты с константной моделью

In [26]:
from sklearn.dummy import DummyClassifier
dummy_model = DummyClassifier(strategy='constant', constant=1)
dummy_model.fit(features_train, target_train)
dummy_pred = dummy_model.predict(features_test)
f1_score(target_test, dummy_pred)

0.3491539413949649

* Лучшая модель - RandomForestClassifier cо следующими гиперпараметрами: глубина - 12, количество деревьев - 170, наилучший результат f1 модель достигла при помощи взвешивания классов. 
* F1-мера 0,6, больше, чем у константной модели.  
* AUC-ROC на тестовой выборке 0,85
* Полнота модели (recall) составляет 0.55, что говорит о том, что модель детектирует на отток 55% клиентов с точностью в 67%. 
* Точность модели (precision) составляет 0.68
* Доля правильных ответов (accuracy) составляет 0.85. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*